In [1]:
import pandas as pd
import numpy as np

In [2]:
#url = "https://data.open-power-system-data.org/household_data/latest/household_data_15min_singleindex.csv"
url= "household_data_15min_singleindex.csv"
df = pd.read_csv(url, sep=",")


In [3]:
df.head(15)
df.describe()

,DE_KN_industrial1_grid_import,DE_KN_industrial1_pv_1,DE_KN_industrial1_pv_2,DE_KN_industrial2_grid_import,DE_KN_industrial2_pv_1,DE_KN_industrial2_storage_charge,DE_KN_industrial2_storage_decharge,DE_KN_industrial3_area_offices,DE_KN_industrial3_area_room_chemistry,DE_KN_industrial3_area_room_clean,...,DE_KN_residential5_grid_import,DE_KN_residential5_refrigerator,DE_KN_residential5_washing_machine,DE_KN_residential6_circulation_pump,DE_KN_residential6_dishwasher,DE_KN_residential6_freezer,DE_KN_residential6_grid_export,DE_KN_residential6_grid_import,DE_KN_residential6_pv,DE_KN_residential6_washing_machine
count,23869.000000,24638.000000,24638.000000,32842.000000,33186.000000,26634.000000,26634.000000,33707.000000,33706.000000,33705.000000,...,43352.000000,40965.000000,40975.000000,35136.000000,28645.000000,26391.000000,26098.000000,33729.000000,36792.000000,27439.000000
mean,152229.804833,4396.266807,6561.568820,5836.910884,13574.777154,666.296668,409.021136,8731.412109,3255.681667,14947.721703,...,1788.844425,232.917891,134.624713,278.668318,30.355847,-190.149631,1200.271647,-4207.670789,5960.096956,25.298370
std,72493.956039,1861.116999,1480.384105,3460.157853,6379.423523,325.693815,197.955154,2773.069760,1143.938678,5036.932613,...,993.140602,143.212621,71.098663,112.254174,19.175891,30.893602,461.521350,2905.702887,3440.338582,15.409011
min,0.750000,0.405000,3001.235000,2.435000,2269.186000,0.000000,0.000000,3759.995000,1277.925000,6089.124000,...,0.020000,0.000000,0.000000,0.002000,0.000000,-267.557000,0.110000,-6314.890000,0.000000,0.000000
25%,110297.820000,3774.850000,6268.160000,3334.656500,7229.420000,393.129750,243.397000,6398.177500,2239.650000,10539.814000,...,968.236500,101.231000,71.948000,212.674000,9.835000,-201.312000,919.577500,-6053.820000,2614.730000,8.836000
50%,169449.508000,5277.900000,7258.130000,5227.990000,15180.538000,734.905000,453.599000,8758.806000,3310.597500,14992.199000,...,1896.180000,229.148000,145.571000,294.252500,32.438000,-181.529000,1500.525000,-5756.100000,6552.671000,24.983000
75%,213027.500000,5733.380000,7552.530000,8295.425000,19688.201000,963.404500,590.974000,11168.252500,4339.840000,19652.850000,...,2535.192500,367.534000,193.392000,347.991750,46.934000,-171.215500,1531.920000,-4606.805000,9427.640000,38.535000
max,248334.016000,5911.535000,7686.045000,13384.365000,20863.631000,1077.885000,646.945000,13424.025000,5040.495000,23093.750000,...,3586.227000,465.621000,248.703000,477.676000,63.243000,1.129000,1537.260000,2404.075000,10107.650000,50.761000


In [4]:
col_names= pd.DataFrame(list(df),columns=['col_name'])
col_names.head()

,col_name
0,utc_timestamp
1,cet_cest_timestamp
2,DE_KN_industrial1_grid_import
3,DE_KN_industrial1_pv_1
4,DE_KN_industrial1_pv_2


In [5]:
def gethousenumber(col_name):
    num_household = list(range(1,7))
    for num in num_household:
        num = str(num)
        if num in col_name:
            return num

In [6]:
col_names['house_number'] = col_names['col_name'].apply(gethousenumber)

In [7]:
col_names['house_split'] = col_names['col_name'].apply(split("_"))

NameError: name 'split' is not defined

In [8]:
prueba = "DE_KN_industrial1_grid_import"
prueba.split("_")

['DE', 'KN', 'industrial1', 'grid', 'import']

In [13]:
col_names[1]

KeyError: 1

In [9]:
col_names.head(15)

,col_name,house_number
0,utc_timestamp,None
1,cet_cest_timestamp,None
2,DE_KN_industrial1_grid_import,1
3,DE_KN_industrial1_pv_1,1
4,DE_KN_industrial1_pv_2,1
5,DE_KN_industrial2_grid_import,2
6,DE_KN_industrial2_pv_1,1
7,DE_KN_industrial2_storage_charge,2
8,DE_KN_industrial2_storage_decharge,2
9,DE_KN_industrial3_area_offices,3


In [19]:
col_names.groupby('house_number').count()
#col_names2=col_names.assign(household=)

,col_name
house_number,
1,12
2,11
3,27
4,9
5,4
6,7
